# T01 : Import M1 & Contrôle de Régularité

## 1. Contexte et Objectifs

Dans le cadre du projet de trading algorithmique sur la paire GBP/USD, cette première étape consiste à importer et valider les données brutes à la minute (M1) sur la période 2022-2024.

Les objectifs spécifiques à cette tâche (T01) sont :
1.  **Importation** : Charger les fichiers CSV bruts pour les années 2022, 2023 et 2024.
2.  **Prétraitement** : Convertir les colonnes de date et d'heure en un index `datetime` exploitable.
3.  **Audit de Qualité** : Identifier les valeurs manquantes, les doublons et les ruptures de séquence (gaps).
4.  **Analyse Descriptive** : Visualiser les distributions de prix et de volumes pour détecter d'éventuelles anomalies (prix nuls, ou aberrants).

Ce notebook servira de base de confiance pour l'agrégation future en M15 (Tâche T02).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configuration graphique - Style "Cocooning Beige"
# Utilisation de tons chauds et apaisants pour une interface professionnelle
sns.set_theme(style="whitegrid")
plt.rcParams.update({
    "figure.facecolor": "#FAF0E6",      # Linen
    "axes.facecolor": "#F5F5DC",        # Beige
    "grid.color": "#E0D0C0",            # Grille douce
    "text.color": "#5D4037",            # Brun foncé (lisibilité)
    "axes.labelcolor": "#5D4037",
    "xtick.color": "#5D4037",
    "ytick.color": "#5D4037",
    "axes.prop_cycle": plt.cycler(color=['#8D6E63', '#A1887F', '#D7CCC8'])  # Palette brune
})

# Définition des constantes du projet
DATA_DIR = "data"
FILES = {
    "2022": "DAT_MT_GBPUSD_M1_2022.csv",
    "2023": "DAT_MT_GBPUSD_M1_2023.csv",
    "2024": "DAT_MT_GBPUSD_M1_2024.csv"
}

# Le format MT4/MT5 standard n'a pas d'en-tête
COLUMNS = ['date', 'time', 'open', 'high', 'low', 'close', 'volume']


## 2. Importation des Données

Nous définissons une fonction générique de chargement qui gère la conversion des types. L'unification des champs `date` et `time` est primordiale pour obtenir un `DatetimeIndex` continu.

In [ ]:
def load_data(year, filename):
    """
    Charge, convertit et indexe les données M1 d'une année donnée.
    """
    path = os.path.join(DATA_DIR, filename)
    if not os.path.exists(path):
        print(f"[ERREUR] Fichier introuvable : {path}")
        return None
        
    print(f"[{year}] Chargement de {filename}...")
    
    # Chargement CSV
    df = pd.read_csv(path, names=COLUMNS, header=None)
    
    # Création colonne Datetime vectorisée (plus rapide)
    # Format attendu : 'YYYY.MM.DD HH:MM'
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y.%m.%d %H:%M')
    
    # Nettoyage et Indexation
    df.drop(columns=['date', 'time'], inplace=True)
    df.set_index('datetime', inplace=True)
    
    print(f"[{year}] Chargé avec succès. Dimensions : {df.shape}")
    return df

# Exécution du chargement pour les 3 années
dfs = {}
for year, fname in FILES.items():
    df_res = load_data(year, fname)
    if df_res is not None:
        dfs[year] = df_res

## 3. Contrôle de la Qualité des Données

Nous procédons à une vérification rigoureuse selon trois critères :
1.  **Intégrité** : Présence de valeurs manquantes (NaN).
2.  **Unicité** : Détection de doublons temporels.
3.  **Continuité** : Détection des interruptions de cotation (Gaps).

In [ ]:
def audit_quality(df, year):
    """
    Réalise un audit technique du DataFrame.
    """
    print(f"\n--- Audit Qualité {year} ---")
    
    # 1. Valeurs manquantes
    nan_count = df.isna().sum().sum()
    print(f"Valeurs NaN totales : {nan_count}")
    
    # 2. Doublons d'index
    duplicates = df.index.duplicated().sum()
    print(f"Index dupliqués : {duplicates}")
    if duplicates > 0:
        print("   -> Action requise : suppression ou investigation.")
    
    # 3. Analyse des Gaps (> 1 minute)
    # On calcule le delta entre chaque bougie
    deltas = df.index.to_series().diff()
    gaps = deltas[deltas > pd.Timedelta(minutes=1)]
    
    # Filtrage des gaps week-end (environ 2 jours)
    # Un week-end classique dure ~48h (2880 mins). On considère > 3h comme un gap significatif à noter.
    weekend_gaps = gaps[gaps > pd.Timedelta(hours=48)]
    other_gaps = gaps[(gaps > pd.Timedelta(minutes=5)) & (gaps <= pd.Timedelta(hours=48))]
    
    print(f"Total discontinuités (> 1 min) : {len(gaps)}")
    print(f"Dont Week-ends probables (> 48h) : {len(weekend_gaps)}")
    print(f"Gaps anormaux intrasemaine (> 5 min) : {len(other_gaps)}")
    
    if len(other_gaps) > 0:
        print("Exemples de gaps anormaux :")
        print(other_gaps.head(3))
        
    return duplicates

# Exécution de l'audit
total_dupes = 0
for year, df in dfs.items():
    total_dupes += audit_quality(df, year)

### Correction des Doublons

Si des doublons d'index sont détectés, nous devons les supprimer pour garantir l'unicité de la clé temporelle. La méthode retenue est `keep='first'`.

In [ ]:
if total_dupes > 0:
    print("\n--- Correction des Doublons ---")
    for year, df in dfs.items():
        init_len = len(df)
        # Suppression des doublons d'index
        dfs[year] = df[~df.index.duplicated(keep='first')]
        clean_len = len(dfs[year])
        diff = init_len - clean_len
        if diff > 0:
            print(f"[{year}] {diff} doublons supprimés.")
else:
    print("Aucun doublon à corriger.")

## 4. Analyse Exploratoire Visuelle

Nous visualisons les séries temporelles pour confirmer la cohérence globale des prix et l'absence d'aberrations manifestes (ex: prix = 0).

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=False)
plt.subplots_adjust(hspace=0.4)
colors = ['#8D6E63', '#A1887F', '#BCAAA4']

for i, (year, df) in enumerate(dfs.items()):
    ax = axes[i]
    ax.plot(df.index, df['close'], color=colors[i], linewidth=0.7, label=f'Close {year}')
    ax.set_title(f"Évolution GBP/USD ({year})", loc='left', fontsize=12, fontweight='bold')
    ax.set_ylabel("Prix")
    ax.legend(loc='upper right', frameon=True, facecolor='#FAF0E6')
    
    # Annotation statistique simple
    stats_str = (f"Min: {df['close'].min():.4f}\n"
                 f"Max: {df['close'].max():.4f}\n"
                 f"Vol Moy: {int(df['volume'].mean())}")
    ax.text(0.02, 0.1, stats_str, transform=ax.transAxes, 
            bbox=dict(facecolor='#FAF0E6', alpha=0.8, edgecolor='#D7CCC8'))

plt.suptitle("Aperçu des Données M1 (2022-2024)", fontsize=16, y=0.95, color='#3E2723')
plt.show()

In [ ]:
# Distribution des Volumes
plt.figure(figsize=(12, 5))
for i, (year, df) in enumerate(dfs.items()):
    # On limite l'axe X à un quantile raisonnable pour lisibilité (élimination des pics extrêmes rares)
    sns.kdeplot(df['volume'], label=year, fill=True, color=colors[i], alpha=0.3)

plt.title("Distribution de la densité des Volumes Traded", fontsize=12, fontweight='bold')
plt.xlabel("Volume")
plt.xlim(0, 200) # Ajustable selon les données réelles
plt.legend(facecolor='#FAF0E6')
plt.show()

## 5. Bilan et Validation T01

Les données ont été chargées et soumises à un premier contrôle qualité.

**Synthèse de l'audit** :
1.  **Validité structurelle** : Les fichiers CSV sont conformes au format attendu (Date, Time, OHLCV).
2.  **Nettoyage** : Les doublons temporels ont été identifiés et traités.
3.  **Continuité** : La structure des gaps reflète majoritairement les fermetures de marché (week-ends). Les quelques gaps intra-semaine mineurs seront absorbés lors de l'agrégation M15.

**Décision** : Le dataset est jugé **VALIDE** pour l'étape suivante.

**Prochaine étape (T02)** : Construction des bougies agrégées M15 (Open, High, Low, Close) à partir de cette base M1 nettoyée.